In [1]:
# 欧拉距离
# distances = []
# for x_train in X_train:
#     d = sqrt(np.sum(x_train - x)**2)
#     distances.append(d)
#  =>
import numpy as np
import matplotlib.pyplot as plt
import xlrd

In [2]:
girl = np.loadtxt("girl.txt")
boy = np.loadtxt("boy.txt")
girlY = np.zeros(girl.shape[0])
boyY = np.ones(boy.shape[0])
X_train = np.vstack((girl, boy))
y_train = np.hstack((girlY, boyY))
x=np.array([156, 44, 36])
from sklearn.

In [3]:
from sklearn.neighbors import KNeighborsClassifier
kNN_classifier = KNeighborsClassifier(n_neighbors = 6)
kNN_classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')

In [5]:
y_predict = kNN_classifier.predict(x.reshape(1,-1))
y_predict

array([0.])

In [10]:
def TN(y_true, y_predict):
    assert len(y_true)==len(y_predict)
    return np.sum(y_true==0 & y_predict==0)

In [11]:
TN(y_test, y_predict)

NameError: name 'y_test' is not defined

In [19]:
knn_clf.fit(X_train, y_train)

KNN(k=6

In [20]:
y_predict = knn_clf.predict(x.reshape(1,-1))

In [22]:
y_predict[0]

0.0